In [1]:
# import the necessary libraries

import datetime, scipy, math
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline

In [2]:
# import the functions from Keras and skLearn libraries

from keras.models import Sequential
from keras.layers import EinsumDense
from keras.layers import LSTMCell
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
# import cvs file

df = pd.read_csv("air_quality.csv", sep=' ')
df.head()

,No,Profile,Tanggal,Jam,O3,CO,NO2,SO2,NO,CO2,VOC,PM1,PM2.5,PM4,PM10,TSP,TEMP,HUM,WS,WD,ISPU,Status,Unnamed: 22
0,1,BANDUNG,2019-05-19,00:04:19,21.35,0.78,6.77,14.06,26.168,409.86,251.202,0.0,0.0,0.0,2.77,0.0,21.4,100.0,0.600,5,9,BAIK,NaN
1,2,BANDUNG,2019-05-19,00:15:01,5.93,0.82,14.87,17.23,53.853,394.79,501.356,0.0,0.0,0.0,11.95,0.0,42.6,200.0,0.000,2,12,BAIK,NaN
2,3,BANDUNG,2019-05-19,00:30:06,6.62,0.63,22.24,2.31,38.304,390.66,753.291,0.0,0.0,0.0,12.71,0.0,63.6,299.2,0.000,2,13,BAIK,NaN
3,4,BANDUNG,2019-05-19,00:49:19,13.29,0.44,19.00,5.32,18.962,401.23,251.043,0.0,0.0,0.0,1.20,0.0,21.3,88.2,0.525,5,6,BAIK,NaN
4,5,BANDUNG,2019-05-19,01:04:19,2.44,0.47,26.74,13.61,35.270,381.67,250.471,0.0,0.0,0.0,3.52,0.0,21.3,97.4,0.975,5,9,BAIK,NaN


In [4]:
# convert str 'date' and 'time' to date python format
df['Datetime'] = pd.to_datetime(df['Tanggal'] + df['Jam'], format='%Y-%m-%d%H:%M:%S')

df = df[['Datetime', 'O3', 'CO', 'NO2', 'SO2', 'NO', 'CO2', 'VOC', 'PM1', 'PM2.5', 'PM4', 'PM10', 'TSP', 'TEMP', 'HUM', 'WS', 'WD', 'ISPU']]

df2 = df.groupby(['Datetime']).mean()

df2

,O3,CO,NO2,SO2,NO,CO2,VOC,PM1,PM2.5,PM4,PM10,TSP,TEMP,HUM,WS,WD,ISPU
Datetime,,,,,,,,,,,,,,,,,
2019-05-19 00:04:19,21.35,0.78,6.77,14.06,26.168,409.86,251.202,0.0,0.0,0.0,2.77,0.0,21.4,100.0,0.600,5.0,9.0
2019-05-19 00:15:01,5.93,0.82,14.87,17.23,53.853,394.79,501.356,0.0,0.0,0.0,11.95,0.0,42.6,200.0,0.000,2.0,12.0
2019-05-19 00:30:06,6.62,0.63,22.24,2.31,38.304,390.66,753.291,0.0,0.0,0.0,12.71,0.0,63.6,299.2,0.000,2.0,13.0
2019-05-19 00:49:19,13.29,0.44,19.00,5.32,18.962,401.23,251.043,0.0,0.0,0.0,1.20,0.0,21.3,88.2,0.525,5.0,6.0
2019-05-19 01:04:19,2.44,0.47,26.74,13.61,35.270,381.67,250.471,0.0,0.0,0.0,3.52,0.0,21.3,97.4,0.975,5.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-18 23:12:37,2.60,0.26,2.82,2.06,83.055,389.93,216.296,0.0,0.0,0.0,12.90,0.0,20.8,68.1,166.875,4.0,13.0
2019-08-18 23:22:37,3.92,0.20,21.88,9.09,48.923,379.97,218.820,0.0,0.0,0.0,5.68,0.0,20.7,68.0,115.425,4.0,6.0
2019-08-18 23:32:37,6.14,0.21,16.31,11.05,58.404,385.59,216.940,0.0,0.0,0.0,15.43,0.0,20.6,68.5,196.725,5.0,15.0


In [5]:
dfdescribe =  df2.describe()
dfdescribe = dfdescribe.transpose()
dfdescribe

,count,mean,std,min,25%,50%,75%,max
O3,9231.0,8.743761,7.249940,0.050,3.920,7.260,11.4400,87.340
CO,9231.0,0.223115,0.294420,0.000,0.070,0.160,0.3000,13.970
NO2,9231.0,16.294503,9.980935,0.060,10.910,15.410,20.4400,443.380
SO2,9231.0,11.778135,6.353669,0.050,7.430,11.500,15.4200,62.640
NO,9231.0,37.358815,15.577667,0.379,26.547,38.683,47.0270,232.100
CO2,9231.0,394.624556,8.881109,379.000,386.910,394.560,402.3400,410.000
VOC,9231.0,258.735153,118.047968,0.000,247.116,250.830,253.1100,2281.690
PM1,9231.0,0.188874,2.458815,0.000,0.000,0.000,0.0000,63.900
PM2.5,9231.0,0.233940,3.052461,0.000,0.000,0.000,0.0000,81.500
PM4,9231.0,0.275431,3.573004,0.000,0.000,0.000,0.0000,92.700


In [6]:
# calculate quantile for each variable

def calculate_quantile(i ,df2):
    Q1 = df2[[i]].quantile(0.25)[0]
    Q3 = df2[[i]].quantile(0.75)[0]
    IQR = Q3 - Q1
    min = df2[[i]].min()[0]
    max = df2[[i]].max()[0]
    min_IQR = Q1 - 1.5*IQR
    max_IQR = Q1 + 1.5*IQR

    return Q1, Q3, min, max, min_IQR, max_IQR

# delete first and last rows to avoid missing values extrapolation
df2.drop(index=[df2.index[0],df2.index[df2.shape[0]-1]], inplace=True)

# find and interpolate the outliers
for i in df2.columns:
    print('\nAtttibute-', i, ":")
    Q1, Q3, min, max, min_IQR, max_IQR = calculate_quantile(i, df2)
    print('Q1 = %.2f' % Q1)
    print('Q3 = %.2f' % Q3)
    print('min IQR = %.2f' % min_IQR)
    print('max IQR = %.2f' % max_IQR)
    if (min < min_IQR):
        print('---> Low outlier is found = %.2f' % min)
    if (max > max_IQR):
        print('---> Max outlier is found = %.2f' % max)

    def convert_nan (x, max_IQR=max_IQR, min_IQR=min_IQR):
        if ((x > max_IQR) | (x < min_IQR)):
            x = np.nan
        else:
            x = x
        return x
    # for HUM
    def convert_nan_HUM (x, max_IQR=100, min_IQR=min_IQR):
        if ((x > max_IQR) | (x < min_IQR)):
            x = np.nan
        else:
            x = x
        return x

    if (i == 'HUM'):
        df2[i] = df2[i].map(convert_nan_HUM)
        df2[i] = df2[i].interpolate(method='linear')


Atttibute- O3 :
Q1 = 3.92
Q3 = 11.39
min IQR = -7.29
max IQR = 15.13
---> Max outlier is found = 87.34

Atttibute- CO :
Q1 = 0.07
Q3 = 0.30
min IQR = -0.27
max IQR = 0.41
---> Max outlier is found = 13.97

Atttibute- NO2 :
Q1 = 10.91
Q3 = 20.44
min IQR = -3.39
max IQR = 25.21
---> Max outlier is found = 443.38

Atttibute- SO2 :
Q1 = 7.43
Q3 = 15.42
min IQR = -4.55
max IQR = 19.41
---> Max outlier is found = 62.64

Atttibute- NO :
Q1 = 26.55
Q3 = 47.03
min IQR = -4.17
max IQR = 57.27
---> Max outlier is found = 232.10

Atttibute- CO2 :
Q1 = 386.91
Q3 = 402.34
min IQR = 363.77
max IQR = 410.05

Atttibute- VOC :
Q1 = 247.12
Q3 = 253.11
min IQR = 238.12
max IQR = 256.11
---> Low outlier is found = 0.00
---> Max outlier is found = 2281.69

Atttibute- PM1 :
Q1 = 0.00
Q3 = 0.00
min IQR = 0.00
max IQR = 0.00
---> Max outlier is found = 63.90

Atttibute- PM2.5 :
Q1 = 0.00
Q3 = 0.00
min IQR = 0.00
max IQR = 0.00
---> Max outlier is found = 81.50

Atttibute- PM4 :
Q1 = 0.00
Q3 = 0.00
min IQR = 0